### Creating Census Tables

Libraries

In [1]:
import re
import pandas as pd
import pickle
import psycopg2
import psycopg2.extras
import os

In [2]:
bcfcsv = 'codebook_batches.csv'
bcf = pd.read_csv(r'D:\\Documents\\data_import\\'+bcfcsv)
varlist = bcf.columns.tolist()
batchproc = bcf['batch'].tolist()
#batchproc = [2] #batch number
dctp = bcf.loc[bcf['batch'].isin(batchproc)].to_dict(orient='records')
print(dctp)

[{'batch': 15, 'desc': 'w tennessee', 'path': 'D:\\Documents\\datafiles\\batch15 - w tenn', 'cbkfilename': 'usa_00026.cbk', 'cbkname': 'cbk15', 'datafile': 'usa_00026.dat', 'cendata': 'cen15', 'cbkcol': 'variable,basis,range,length', 'pattern': '\\s{2}(\\w+)\\s+([HP])\\s+([0-9,-]+)\\s+(\\d+)'}]


In [3]:
def open_database(database):
    conn = psycopg2.connect(dbname=database, user='postgres', password = '3701', host='localhost', port='5432')
    return conn

def create_cursor():
    cursor = conn.cursor()
    return cursor

def set_census_schema():
    cursor.execute("SET SCHEMA 'production'")

def close_database():
    #insert rollback logic
    print('committing the records')
    conn.commit()
    conn.close() 

In [4]:
def read_codebook(cbkpickle):
    cbdf = pd.read_pickle(cbkpickle)
    return cbdf

def create_column_list(cbdf):
    x = cbdf['length'].astype('str')
    cbdf['column_name'] = cbdf['variable'].str.lower() + ' char('+x+')'
    column_list = cbdf['column_name'].tolist()
    return column_list

def create_column_text(cbdf):
    x = cbdf['length'].astype('str')
    cbdf['column_name'] = cbdf['variable'].str.lower() + ' char('+x+')'
    column_list = cbdf['column_name'].tolist()
    columns_text = '(' + ', '.join(column_list) + ')'
    return columns_text

In [5]:
def create_table(tablename, columns):
    columns_text = '(' + ', '.join(columns) + ');'
    cursor.execute('DROP TABLE IF EXISTS ' + tablename + ';')
    cursor.execute('CREATE TABLE ' + tablename + ' ' + columns_text+';')
    conn.commit()

def clear_table(tablename):
    clear = 'DELETE FROM ' + tablename

def get_column_counts(tablename, columns):
    qry_cc = 'select count(*) from information_schema.columns where table_name =' + "'"+ tablename + "'"+';'
    #col_ct_sql = cursor.execute(qry_cc)
    #col_ct_list = len(columns)+1
    #print('columns in table:',col_ct_sql,'\n','columns in list:',col_ct_list)

In [6]:
conn = open_database('thesis')
cursor = create_cursor()
set_census_schema()

In [7]:
for bat in dctp:
    sd = bat
    tablename = sd['cendata']
    cbkpickle = sd['path']+'\\'+sd['cbkname']+'.pickle'
    cbdf = read_codebook(cbkpickle)
    column_list = create_column_list(cbdf)
    create_table(tablename,column_list)
    print('Created '+tablename)

Created cen15


In [8]:
conn.close()